In [ ]:
!pip install tensorflow
!pip install tflearn

In [1]:
# things we need for NLP
import nltk
#from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import SnowballStemmer
#stemmer = LancasterStemmer()
import pandas as pd

# things we need for Tensorflow
import numpy as np
#import tflearn
import tensorflow as tf
import random
import pandas as pd
import json

In [6]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [ ]:
nltk.download('punkt')

In [3]:
stemmer = SnowballStemmer('english')

In [5]:
# Load Dataset
import json
with open('intents_eng_update.json', 'r') as json_data:
  intents = json.load(json_data)

# Load yoruba dataset
with open('intent_yoruba_update.json', 'r') as json_data:
  intents_yo = json.load(json_data)

# Load hindi dataset
with open('intent_hindi_update.json', 'r') as json_data:
  intents_hi = json.load(json_data)

In [ ]:
#With our intents JSON file loaded, we can now begin to organize our documents, words and classification classes.
documents = []
classes = []
words  = []
ignore_words = ['?']

#loop through each sentence in our intent file pattern in english dataset
for intent in intents['intents']:
  for pattern in intent['patterns']:
    #tokenize each word in the pattern
    w = nltk.word_tokenize(pattern)
    #add to our words list
    words.extend(w)
    #add to document in our corpus
    documents.append((w, intent['tag']))
    #add to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])


#loop through each sentence in our intent file pattern yoruba dataset
for intent in intents_yo['intents']:
  for pattern in intent['patterns']:
    #tokenize each word in the pattern
    w = nltk.word_tokenize(pattern)
    #add to our words list
    words.extend(w)
    #add to document in our corpus
    documents.append((w, intent['tag']))
    #add to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])


#loop through each sentence in our intent file pattern hindi dataset
for intent in intents_hi['intents']:
  for pattern in intent['patterns']:
    #tokenize each word in the pattern
    w = nltk.word_tokenize(pattern)
    #add to our words list
    words.extend(w)
    #add to document in our corpus
    documents.append((w, intent['tag']))
    #add to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

#stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

#remove duplicates
classes = sorted(list(set(classes)))

print(len(documents), 'documents')
print(len(classes), 'classes', classes)
print(len(words), 'unique stemmed words', words)

In [ ]:
#creating our training data
training = []
output = []
#create an empty array for our output
output_empty = [0] * len(classes)

#training set, bag of words for each sentence
for doc in documents:
  #initialize our bag of words
  bag = []
  #list of tokenized words for the pattern
  pattern_words = doc[0]
  # stem each word
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  # create our bag od words array
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [8]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.2, random_state=10)

In [9]:
train_x = np.array(train_x)
train_y = np.array(train_y)
test_x=np.array(test_x)
test_y = np.array(test_y)

In [ ]:
# Define the model
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Dense, Flatten, LSTM
from nltk import metrics
vocab_size = len(words)
model = keras.Sequential()
e = Embedding(vocab_size, 200, input_length=len(train_x[0]), trainable=False)
model.add(e)
model.add(LSTM(128))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile the model
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Summarize the model
print(model.summary())

In [ ]:
# Fit the model
history = model.fit(train_x, train_y, epochs=50, batch_size=16)

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_x, test_y)
print("Loss:", loss)
print("Accuracy:", accuracy)